In [32]:
import numpy as np
from decimal import Decimal
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

def cartesian(arrays):

    """[[numpy 배열], [numpy 배열]...] 의 형태를 받아 배열 끼리의 데카르트 곱을 numpy 배열로 반환"""

    la = len(arrays)
    dtype = np.result_type(*arrays)
    arr = np.empty([la] + [len(a) for a in arrays], dtype=dtype)
    for i, a in enumerate(np.ix_(*arrays)):
        arr[i, ...] = a
    return arr.reshape(la, -1).T

def equation(Tox, muns, Vt, m, Vds, Vgs, W=Decimal('1e-6'), L=Decimal('1e-6'), Eox=Decimal('3.9')*Decimal('8.85e-12')):
    Coxe = Eox/Tox
    if Vds < (Vgs - Vt)/m:
        return W/L * Coxe * muns * (Vgs-Vt-m/2*Vds)*Vds
    elif Vds >= (Vgs - Vt)/m:
        return W/(2*m*L) * Coxe * muns * (Vgs - Vt)*(Vgs - Vt)
    elif Vgs < Vt:
        return 0
    else:
        raise Exception('방정식 예외 발생')
    
def equationans(Tox, muns, Vt, m):
    temp = cartesian((np.arange(0, 5), np.arange(0, 5)))
    return [float(equation(Tox, muns, Vt, m, *x)) for x in temp]

data = pd.read_csv('data.csv')
ans = pd.read_csv('ans.csv')

midTox = Decimal('1e-8')
midmuns = Decimal('0.02')
midm = Decimal('1.2')
midVt = Decimal('0.7')

scaler = preprocessing.MinMaxScaler()
data = scaler.fit_transform(data)

x = ans
y = data

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, test_size=0.2)

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model
import keras

inputs = Input(shape=(25,))

x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(4)(x)

In [ ]:
import keras.callbacks

callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=3
    ),
    keras.callbacks.ModelCheckpoint(
        filepath='checkpoint.keras',
        monitor='val_loss',
        save_best_only=True
    )
]

model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer=keras.optimizers.Adam(),
              loss='mse',
              metrics=['accuracy'])
history = model.fit(x_test, y_test, epochs=20, callbacks=callbacks_list, validation_data=(x_test, y_test))

In [ ]:
import keras.models

model = keras.models.load_model('checkpoint.keras')

In [ ]:
import matplotlib.pyplot as plt

def vis(history,name) :
    plt.title(f"{name.upper()}")
    plt.xlabel('epochs')
    plt.ylabel(f"{name.lower()}")
    value = history.history.get(name)
    val_value = history.history.get(f"val_{name}",None)
    epochs = range(1, len(value)+1)
    plt.plot(epochs, value, 'b-', label=f'training {name}')
    if val_value is not None :
        plt.plot(epochs, val_value, 'r:', label=f'validation {name}')
    plt.legend(loc='upper center', bbox_to_anchor=(0.05, 1.2) , fontsize=10 , ncol=1)
    
def plot_history(history) :
    key_value = list(set([i.split("val_")[-1] for i in list(history.history.keys())]))
    plt.figure(figsize=(12, 4))
    for idx , key in enumerate(key_value) :
        plt.subplot(1, len(key_value), idx+1)
        vis(history, key)
    plt.tight_layout()
    plt.show()

plot_history(history)

In [ ]:
import joblib

joblib.dump(model, 'deepmodel.pkl')

In [ ]:
test = [midTox * Decimal('0.7'), midmuns * Decimal('1.2'), midm * Decimal('1.06'), midVt * Decimal('0.96')]

pred = model.predict([equationans(*test)])
pred = scaler.inverse_transform(pred)
print(test, pred)